In [ ]:
import pandas as pd
import pandas as pd 
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/colab/FinalDataset/FinalTrainAbleDataset.xlsx").sample(frac=1)
df.columns

Index(['Unnamed: 0', 'tags', 'headline', 'url', 'clean_body',
       'stop_clean_body', 'target'],
      dtype='object')

In [ ]:
df.head(2)

,Unnamed: 0,tags,headline,url,clean_body,stop_clean_body,target
4197,326,"['মতামত', 'আন্তর্জাতিক', 'লেখকের কলাম']",যুক্তরাষ্ট্রে ইবোলা,https://www.prothomalo.com/opinion/column/যুক্...,টমাস এরিক ডানকানই প্রথম মার্কিন যুক্তরাষ্ট্রে...,টমাস এরিক ডানকানই মার্কিন যুক্তরাষ্ট্রে ইবোলা...,column
3840,106,['আমার চট্টগ্রাম'],আনোয়ারায় স্কুল ব্যাংকিং শুরু,https://www.prothomalo.com/bangladesh/district...,আনোয়ারায় শিক্ষার্থীদের মধ্যে স্কুল ব্যাংকিং...,আনোয়ারায় শিক্ষার্থীদের স্কুল ব্যাংকিং কার্যক...,district


In [ ]:
df = df[['stop_clean_body','target']] 

In [ ]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

In [ ]:
def label_sentences(corpus, label_type):
  labeled = []
  for i,v in enumerate(corpus):
    label = label_type + '_' +str(i)
    labeled.append(TaggedDocument(v.split(),[label]))
  return labeled


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.stop_clean_body, df.target, random_state= 0 , test_size = 0.2)  

In [ ]:
X_train.shape, X_test.shape

((7072,), (1768,))

In [ ]:
X_train = label_sentences(X_train, 'Train')
X_test = label_sentences(X_test, 'Test')

In [ ]:
len(X_train )

7072

In [ ]:
all_data = X_train + X_test

In [ ]:
#################### Doc2VecDesign #####################
import multiprocessing
cores = multiprocessing.cpu_count()
from sklearn import utils
model_dbow = Doc2Vec(dm=0,vector_size = 300, negative = 5, min_count =1, alpha = 0.065, min_alpha = 0.065,workers=cores)
model_dbow.build_vocab([x for x in tqdm(all_data)])
for epoch in range(50):
    model_dbow.train(utils.shuffle([x for x in tqdm(all_data)]), total_examples=len(all_data), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha 

100%|██████████| 8840/8840 [00:00<00:00, 975111.70it/s]


In [ ]:
from gensim.test.utils import get_tmpfile
fname = get_tmpfile("/content/drive/MyDrive/colab/SavedModel/WithOutStopDoc2VecModel")
model_dbow.save(fname)

In [ ]:
# from gensim.models import Doc2Vec
# model_dbow = Doc2Vec.load('/content/drive/MyDrive/colab/SavedModel/WithOutStopDoc2VecModel')

In [ ]:
import numpy as np

def get_vectors(model,corpus_size,vectors_size,vectors_type):
    vectors = np.zeros((corpus_size,vectors_size))
    for i in range(0,corpus_size):
      prefix = vectors_type + '_' + str(i)
      vectors[i] = model.docvecs[prefix]
    return vectors

In [ ]:
train_vectors_dbow = get_vectors(model_dbow,len(X_train), 300, "Train")
test_vectors_dbow = get_vectors(model_dbow,len(X_test), 300, "Test")

In [ ]:
len(X_train)

7072

In [ ]:
print('################### Doc2vec Logistic Regression ####################')
from sklearn.metrics import accuracy_score, f1_score,classification_report
model = LogisticRegression()
model = model.fit(train_vectors_dbow, y_train)
pred = model.predict(test_vectors_dbow)
print('Validation accuracy {:.2f}%'.format(accuracy_score(pred, y_test)*100))
print(classification_report(y_test,pred))

################### Doc2vec Logistic Regression ####################
Validation accuracy 82.41%
               precision    recall  f1-score   support

         asia       0.95      0.86      0.90        85
   bangladesh       0.41      0.33      0.37       103
     business       0.83      0.80      0.81       108
       column       0.95      0.97      0.96       102
      cricket       0.90      0.89      0.89       108
        crime       0.72      0.64      0.68       118
     district       0.74      0.88      0.81       111
  durporobash       0.93      0.97      0.95        98
    education       0.93      0.88      0.90        85
entertainment       0.87      0.91      0.89       102
     football       0.88      0.75      0.81       122
        india       0.96      0.97      0.96       110
         life       0.74      0.75      0.74       106
     politics       0.74      0.79      0.76       102
 science-tech       0.87      0.94      0.90        90
       sports       0.7

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
print('################### Doc2vec SGD Classifier ####################')
from sklearn.metrics import accuracy_score, f1_score,classification_report
from sklearn.linear_model import SGDClassifier
model = SGDClassifier(max_iter=1000, tol=1e-3,n_jobs=2,penalty="l2")
model = model.fit(train_vectors_dbow, y_train)
pred = model.predict(test_vectors_dbow)
print('Validation accuracy {:.2f}%'.format(accuracy_score(pred, y_test)*100))
print(classification_report(y_test,pred)) 

################### Doc2vec SGD Classifier ####################
Validation accuracy 78.17%
               precision    recall  f1-score   support

         asia       0.89      0.84      0.86        85
   bangladesh       0.38      0.31      0.34       103
     business       0.87      0.69      0.77       108
       column       0.86      1.00      0.92       102
      cricket       0.90      0.85      0.88       108
        crime       0.68      0.58      0.63       118
     district       0.64      0.86      0.73       111
  durporobash       0.88      0.94      0.91        98
    education       0.90      0.88      0.89        85
entertainment       0.91      0.84      0.87       102
     football       0.82      0.80      0.80       122
        india       0.88      0.99      0.93       110
         life       0.80      0.56      0.66       106
     politics       0.78      0.74      0.76       102
 science-tech       0.77      0.94      0.85        90
       sports       0.71    